In [4]:
import pandas as pd
import numpy as np
import os

In [9]:
nl_1k = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_nolate_1k.csv")
nl_all = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_nolate_all.csv")
yl_1k = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_yeslate_1k.csv")
yl_all = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_yeslate_all.csv")

In [32]:
df = pd.DataFrame()
df['username'] = nl_1k['username']
df['mean_nl_1k'] = nl_1k.iloc[:,1:].mean(axis=1)
df['mean_nl_all'] = nl_all.iloc[:,1:].mean(axis=1)
df['mean_yl_1k'] = yl_1k.iloc[:,1:].mean(axis=1)
df['mean_yl_all'] = yl_all.iloc[:,1:].mean(axis=1)

rank_df = pd.DataFrame()
for col in df.columns[1:]:
    rank_df[col] = df[col].rank()
rank_df = rank_df.add_suffix('_rank')
print(rank_df)
frames = [df, rank_df]
merged_df = pd.concat(frames)

print(merged_df.head(5))

merged_df.to_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_collection.csv", index = False)




     mean_nl_1k_rank  mean_nl_all_rank  mean_yl_1k_rank  mean_yl_all_rank
0              204.0             221.0            203.0             222.0
1              160.0             172.0            160.0             151.0
2              144.0             136.0            140.0             130.0
3               68.0              65.0             53.0              52.0
4               82.0              77.0             80.0              80.0
..               ...               ...              ...               ...
651            269.0             272.0            157.0             181.0
652             37.0              51.0             36.0              44.0
653            490.0             486.0            553.0             551.0
654             99.0             119.0            124.0             129.0
655            467.0             454.0            501.0             489.0

[656 rows x 4 columns]
          username  mean_nl_1k  mean_nl_all  mean_yl_1k  mean_yl_all  \
0              j